In [ ]:
!pip install transformers datasets torch gradio

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
from datasets import load_dataset

# Load the IMDB dataset
dataset = load_dataset('imdb')
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [ ]:
# Print the first example from the training set
print(dataset['train'][0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:
from transformers import AutoTokenizer

model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def tokenize_function(example):
    return tokenizer(example['text'], padding='max_length', truncation=True, max_length=128)

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
train_dataset = tokenized_dataset['train']
val_dataset = tokenized_dataset['test']

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none"  # Disable all external logging (including W&B)
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Start training
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.337100,0.325054
2,0.232900,0.414858
3,0.114300,0.546293


TrainOutput(global_step=9375, training_loss=0.2450000419108073, metrics={'train_runtime': 2772.457, 'train_samples_per_second': 27.052, 'train_steps_per_second': 3.381, 'total_flos': 4933332288000000.0, 'train_loss': 0.2450000419108073, 'epoch': 3.0})

In [ ]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.546293318271637, 'eval_runtime': 179.1957, 'eval_samples_per_second': 139.512, 'eval_steps_per_second': 17.439, 'epoch': 3.0}


In [ ]:
from transformers import pipeline

# Create a sentiment-analysis pipeline using the trained model
sentiment_pipeline = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

# Test the model with custom inputs
print(sentiment_pipeline("I absolutely loved this movie!"))
print(sentiment_pipeline("This was the worst experience ever."))
print(sentiment_pipeline("It was an okay experience, not too great, not too bad."))

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_1', 'score': 0.9994377493858337}]
[{'label': 'LABEL_0', 'score': 0.9976224303245544}]
[{'label': 'LABEL_1', 'score': 0.9902799725532532}]


In [ ]:
import gradio as gr

def sentiment_analysis(text):
    result = sentiment_pipeline(text)
    label_map = {
        'LABEL_0': 'Negative',
        'LABEL_1': 'Positive'
    }
    label = label_map.get(result[0]['label'], 'Unknown')
    score = result[0]['score']
    return f"{label} with a confidence score of {score:.2f}"

interface = gr.Interface(
    fn=sentiment_analysis,
    inputs="text",
    outputs="text",
    title="Sentiment Analysis with BERT",
    description="Enter a movie review, and the model will predict whether it's positive or negative."
)

# Launch the interface
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d2ee24bcbd1362dcd9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [ ]:
!zip -r saved_model.zip saved_model/

  adding: saved_model/ (stored 0%)
  adding: saved_model/model.safetensors (deflated 7%)
  adding: saved_model/special_tokens_map.json (deflated 42%)
  adding: saved_model/tokenizer.json (deflated 71%)
  adding: saved_model/tokenizer_config.json (deflated 76%)
  adding: saved_model/vocab.txt (deflated 53%)
  adding: saved_model/config.json (deflated 49%)


In [ ]:
from google.colab import files
files.download('saved_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>